In [1]:
from jijmodelingCompiler.para_search import para_search_solver as jm_compiler_para_search
from jijmodeling_engine import para_search_solver as jm_engine_para_search

In [14]:
import jijzept as jz
import jijmodeling as jm
import openjij as oj
import dimod

def simple_problem():
    d = jm.Placeholder("d", dim=1)
    x = jm.Binary("x", shape=(d.shape[0].set_latex("n")))
    i = jm.Element("i", d.shape[0])
    problem = jm.Problem("simple_problem")
    problem += jm.Sum(i, d[i] * x[i])
    problem += jm.Constraint("onehot", jm.Sum(i, x[i]) == 1)
    return problem

problem = simple_problem()
d_p = [-1, -1, -1, -8, -1, -1]
instance_data = {"d": d_p}

def sample_qubo(sampler, num_reads, num_sweeps):
    return lambda q: sampler.sample_qubo(q, num_reads=num_reads, num_sweeps=num_sweeps)

def evaluate(sampleset_list, problem, instance_data):
    sampleset = dimod.concatenate(sampleset_list)
    decoded_samples = problem.decode(sampleset, instance_data)
    return sampleset, decoded_samples

def solve_by_jm_compiler_para_search(sampler, problem, instance_data, num_repeats, num_reads=1, num_sweeps=1000):
    sampleset_list, best_prams = jm_compiler_para_search(
        problem, instance_data, num_repeats=num_repeats, sample_qubo_callback=sample_qubo(sampler, num_reads=num_reads, num_sweeps=num_sweeps)
    )
    return evaluate(sampleset_list, problem, instance_data)

def solve_by_jm_engine_para_search(sampler, problem, instance_data, num_repeats, num_reads=1, num_sweeps=1000):
    sampleset_list, best_prams = jm_engine_para_search(
        problem, instance_data, num_repeats=num_repeats, sample_qubo_callback=sample_qubo(sampler, num_reads=num_reads, num_sweeps=num_sweeps)
    )
    return evaluate(sampleset_list, problem, instance_data)

In [18]:
import jijbench as jb

bench = jb.Benchmark(
    params={"sampler": [oj.SASampler()], "num_reads": [5], "num_repeats": [5, 10, 15]},
    solver=[solve_by_jm_compiler_para_search, solve_by_jm_engine_para_search],
    problem=problem,
    instance_data=instance_data
)
bench.run()

In [20]:
bench.table[["solver", "sampler", "num_repeats", "objective", "num_occurances", "num_feasible", "num_samples"]]

,solver,sampler,num_repeats,objective,num_occurances,num_feasible,num_samples
0,solve_by_jm_compiler_para_search,<openjij.sampler.sa_sampler.SASampler object a...,5.0,"[-13.0, -13.0, -13.0, -13.0, -13.0, 0.0, 0.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5,20
1,solve_by_jm_compiler_para_search,<openjij.sampler.sa_sampler.SASampler object a...,10.0,"[-13.0, -13.0, -13.0, -13.0, -13.0, 0.0, 0.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5,20
2,solve_by_jm_compiler_para_search,<openjij.sampler.sa_sampler.SASampler object a...,15.0,"[-13.0, -13.0, -13.0, -13.0, -13.0, 0.0, 0.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5,20
3,solve_by_jm_engine_para_search,<openjij.sampler.sa_sampler.SASampler object a...,5.0,"[-13.0, -13.0, -13.0, -13.0, -13.0, 0.0, 0.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",5,15
4,solve_by_jm_engine_para_search,<openjij.sampler.sa_sampler.SASampler object a...,10.0,"[-13.0, -13.0, -13.0, -13.0, -13.0, 0.0, 0.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",5,15
5,solve_by_jm_engine_para_search,<openjij.sampler.sa_sampler.SASampler object a...,15.0,"[-13.0, -13.0, -13.0, -13.0, -13.0, 0.0, 0.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",5,15
